In [ ]:
#本代码为模型训练部分
#
#本代码分为三部分，分别是数据预处理、构建神经网络和学习过程
#
#在数据预处理部分首先使用xlrd从excel中读入数据，再将数据归一化处理后转化成列表形式；
#最后将列表形式的数据转化为pytorch的tensor格式，并且根据网络结构的要求调整矩阵形状。
#
#在构建网络部分构建了一个包含三个隐藏层、三个非线性层和一个全连接层的神经网络，并且采用relu函数作为非线性激活函数
#
#在学习过程中使用梯度下降的方式进行回溯，调参得到的超参为学习次数50000次，学习率0.05

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import xlrd

#打开excel文件
datas=xlrd.open_workbook("./data.xlsx")
datasheet=datas.sheet_by_index(0)

#数据预处理（相同处理方式仅在第一个参数处作注释，其余同理）
#
#参数Age_08_04
Age=datasheet.col_values(3)
Age_int=[]
Age_max=0
Age_min=100
for i in range(1,len(Age)):#从excel中读取数据
    k=float(Age[i])
    Age_int.append(k)
    if k<Age_min:
        Age_min=k
    if k>Age_max:
        Age_max=k
Age_less=Age_max-Age_min#计算极差
for i in range(0,len(Age_int)):#归一化处理
    Age_int[i]=2*((Age_int[i]-Age_min)/Age_less)-1

#参数Mfg_Month
Mont=datasheet.col_values(4)
Mont_int=[]
Mont_max=0
Mont_min=100000
for i in range(1,len(Mont)):
    k=float(Mont[i])
    Mont_int.append(k)
    if k<Mont_min:
        Mont_min=k
    if k>Mont_max:
        Mont_max=k
Mont_less=Mont_max-Mont_min
for i in range(0,len(Mont_int)):
    Mont_int[i]=2*((Mont_int[i]-Mont_min)/Mont_less)-1

#参数Mfg_Year
Year=datasheet.col_values(5)
Year_int=[]
Year_max=0
Year_min=100000
for i in range(1,len(Year)):
    k=float(Year[i])
    Year_int.append(k)
    if k<Year_min:
        Year_min=k
    if k>Year_max:
        Year_max=k
Year_less=Year_max-Year_min
for i in range(0,len(Year_int)):
    Year_int[i]=2*((Year_int[i]-Year_min)/Year_less)-1

#参数KM
KM=datasheet.col_values(6)
KM_int=[]
KM_max=0
KM_min=100000
for i in range(1,len(KM)):
    k=float(KM[i])
    KM_int.append(k)
    if k<KM_min:
        KM_min=k
    if k>KM_max:
        KM_max=k
KM_less=KM_max-KM_min
for i in range(0,len(KM_int)):
    KM_int[i]=2*((KM_int[i]-KM_min)/KM_less)-1

#参数Fuel_Type
Type=datasheet.col_values(7)
Type_int=[]
for i in range(1,len(Type)):#参数为三个离散值，将其分别赋值为-0.5、0、0.5
    if Type[i]=='Diesel':
        k=0.5
    elif Type[i]=='Petrol':
        k=-0.5
    else:
        k=0.0
    Type_int.append(k)

#参数HP
HP=datasheet.col_values(8)
HP_int=[]
HP_max=0
HP_min=100000
for i in range(1,len(HP)):
    k=float(HP[i])
    HP_int.append(k)
    if k<HP_min:
        HP_min=k
    if k>HP_max:
        HP_max=k
HP_less=HP_max-HP_min
for i in range(0,len(HP_int)):
    HP_int[i]=2*((HP_int[i]-HP_min)/HP_less)-1

#参数Met_Color
MetC=datasheet.col_values(9)
MetC_int=[]
MetC_max=0
MetC_min=100000
for i in range(1,len(MetC)):
    k=float(MetC[i])
    MetC_int.append(k)
    if k<MetC_min:
        MetC_min=k
    if k>MetC_max:
        MetC_max=k
MetC_less=MetC_max-MetC_min
for i in range(0,len(MetC_int)):
    MetC_int[i]=2*((MetC_int[i]-MetC_min)/MetC_less)-1

#参数Automatic
Auto=datasheet.col_values(11)
Auto_int=[]
Auto_max=0
Auto_min=100000
for i in range(1,len(Auto)):
    k=float(Auto[i])
    Auto_int.append(k)
    if k<Auto_min:
        Auto_min=k
    if k>Auto_max:
        Auto_max=k
Auto_less=Auto_max-Auto_min
for i in range(0,len(Auto_int)):
    Auto_int[i]=2*((Auto_int[i]-Auto_min)/Auto_less)-1

#参数CC
CC=datasheet.col_values(12)
CC_int=[]
CC_max=0
CC_min=100000
for i in range(1,len(CC)):
    k=float(CC[i])
    CC_int.append(k)
    if k<CC_min:
        CC_min=k
    if k>CC_max:
        CC_max=k
CC_less=CC_max-CC_min
for i in range(0,len(CC_int)):
    CC_int[i]=2*((CC_int[i]-CC_min)/CC_less)-1

#参数Doors
Doors=datasheet.col_values(13)
Doors_int=[]
Doors_max=0
Doors_min=100000
for i in range(1,len(Doors)):
    k=float(Doors[i])
    Doors_int.append(k)
    if k<Doors_min:
        Doors_min=k
    if k>Doors_max:
        Doors_max=k
Doors_less=Doors_max-Doors_min
for i in range(0,len(Doors_int)):
    Doors_int[i]=2*((Doors_int[i]-Doors_min)/Doors_less)-1

#参数Gears
Gears=datasheet.col_values(15)
Gears_int=[]
Gears_max=0
Gears_min=100000
for i in range(1,len(Gears)):
    k=float(Gears[i])
    Gears_int.append(k)
    if k<Gears_min:
        Gears_min=k
    if k>Gears_max:
        Gears_max=k
Gears_less=Gears_max-Gears_min
for i in range(0,len(Gears_int)):
    Gears_int[i]=2*((Gears_int[i]-Gears_min)/Gears_less)-1

#参数Quarterly_Tax
Quart=datasheet.col_values(16)
Quart_int=[]
Quart_max=0
Quart_min=100000
for i in range(1,len(Quart)):
    k=float(Quart[i])
    Quart_int.append(k)
    if k<Quart_min:
        Quart_min=k
    if k>Quart_max:
        Quart_max=k
Quart_less=Quart_max-Quart_min
for i in range(0,len(Quart_int)):
    Quart_int[i]=2*((Quart_int[i]-Quart_min)/Quart_less)-1

#参数Weight
Weight=datasheet.col_values(17)
Weight_int=[]
Weight_max=0
Weight_min=100000
for i in range(1,len(Weight)):
    k=float(Weight[i])
    Weight_int.append(k)
    if k<Weight_min:
        Weight_min=k
    if k>Weight_max:
        Weight_max=k
Weight_less=Weight_max-Weight_min
for i in range(0,len(Weight_int)):
    Weight_int[i]=2*((Weight_int[i]-Weight_min)/Weight_less)-1

#参数Mfg_Guarantee
Mfg=datasheet.col_values(18)
Mfg_int=[]
Mfg_max=0
Mfg_min=100000
for i in range(1,len(Mfg)):
    k=float(Mfg[i])
    Mfg_int.append(k)
    if k<Mfg_min:
        Mfg_min=k
    if k>Mfg_max:
        Mfg_max=k
Mfg_less=Mfg_max-Mfg_min
for i in range(0,len(Mfg_int)):
    Mfg_int[i]=2*((Mfg_int[i]-Mfg_min)/Mfg_less)-1

#参数BOVAG_Guarantee
Bovar=datasheet.col_values(19)
Bovar_int=[]
Bovar_max=0
Bovar_min=100000
for i in range(1,len(Bovar)):
    k=float(Bovar[i])
    Bovar_int.append(k)
    if k<Bovar_min:
        Bovar_min=k
    if k>Bovar_max:
        Bovar_max=k
Bovar_less=Bovar_max-Bovar_min
for i in range(0,len(Bovar_int)):
    Bovar_int[i]=2*((Bovar_int[i]-Bovar_min)/Bovar_less)-1

#参数Guarantee_Period
Guarantee=datasheet.col_values(20)
Guarantee_int=[]
Guarantee_max=0
Guarantee_min=100000
for i in range(1,len(Guarantee)):
    k=float(Guarantee[i])
    Guarantee_int.append(k)
    if k<Guarantee_min:
        Guarantee_min=k
    if k>Guarantee_max:
        Guarantee_max=k
Guarantee_less=Guarantee_max-Guarantee_min
for i in range(0,len(Guarantee_int)):
    Guarantee_int[i]=2*((Guarantee_int[i]-Guarantee_min)/Guarantee_less)-1

#参数ABS
ABS=datasheet.col_values(21)
ABS_int=[]
for i in range(1,len(ABS)):#参数数据值只有0和1，故省略归一化处理
    k=float(ABS[i])
    ABS_int.append(k)

#参数Airbag_1
A1=datasheet.col_values(22)
A1_int=[]
for i in range(1,len(A1)):
    k=float(A1[i])
    A1_int.append(k)

#参数Airbag_2
A2=datasheet.col_values(23)
A2_int=[]
for i in range(1,len(A2)):
    k=float(A2[i])
    A2_int.append(k)

#参数Airco
Airco=datasheet.col_values(24)
Airco_int=[]
for i in range(1,len(Airco)):
    k=float(Airco[i])
    Airco_int.append(k)

#参数Automatic_airco
Automatic2=datasheet.col_values(25)
Automatic2_int=[]
for i in range(1,len(Automatic2)):
    k=float(Automatic2[i])
    Automatic2_int.append(k)

#参数Boardcomputer
Board=datasheet.col_values(26)
Board_int=[]
for i in range(1,len(Board)):
    k=float(Board[i])
    Board_int.append(k)

#参数CD_Player
CD=datasheet.col_values(27)
CD_int=[]
for i in range(1,len(CD)):
    k=float(CD[i])
    CD_int.append(k)

#参数Central_Lock
Central=datasheet.col_values(28)
Central_int=[]
for i in range(1,len(Central)):
    k=float(Central[i])
    Central_int.append(k)

#参数Powered_Windows
Powered=datasheet.col_values(29)
Powered_int=[]
for i in range(1,len(Powered)):
    k=float(Powered[i])
    Powered_int.append(k)

#参数Power_Steering
Power=datasheet.col_values(30)
Power_int=[]
for i in range(1,len(Power)):
    k=float(Power[i])
    Power_int.append(k)

#参数Radio
Radio=datasheet.col_values(31)
Radio_int=[]
for i in range(1,len(Radio)):
    k=float(Radio[i])
    Radio_int.append(k)

#参数Mistlamps
Mistla=datasheet.col_values(32)
Mistla_int=[]
for i in range(1,len(Mistla)):
    k=float(Mistla[i])
    Mistla_int.append(k)

#参数Sport_Model
Sport=datasheet.col_values(33)
Sport_int=[]
for i in range(1,len(Sport)):
    k=float(Sport[i])
    Sport_int.append(k)

#参数Backseat_Divider
Back=datasheet.col_values(34)
Back_int=[]
for i in range(1,len(Back)):
    k=float(Back[i])
    Back_int.append(k)

#参数Metallic_Rim
Metall=datasheet.col_values(35)
Metall_int=[]
for i in range(1,len(Metall)):
    k=float(Metall[i])
    Metall_int.append(k)

#参数Radio_cassette
Radio2=datasheet.col_values(36)
Radio2_int=[]
for i in range(1,len(Radio2)):
    k=float(Radio2[i])
    Radio2_int.append(k)

#参数Tow_Bar
Tow=datasheet.col_values(37)
Tow_int=[]
for i in range(1,len(Tow)):
    k=float(Tow[i])
    Tow_int.append(k)

#参数Validation
Validation=datasheet.col_values(38)
Validation_int=[]
for i in range(1,len(Validation)):
    k=float(Validation[i])
    Validation_int.append(k)

#合并数据为张量
X_tmp=torch.tensor([Age_int,Mont_int,Year_int,KM_int,Type_int,HP_int,MetC_int,Auto_int,CC_int,Doors_int,Gears_int,Quart_int,Weight_int,Mfg_int,Bovar_int,Guarantee_int,ABS_int,A1_int,A2_int,Airco_int,Automatic2_int,Board_int,CD_int,Central_int,Powered_int,Power_int,Radio_int,Mistla_int,Sport_int,Back_int,Metall_int,Radio2_int,Tow_int,Validation_int])
x=X_tmp.permute(1,0)#矩阵转置，适配网络输入端

#处理样本数据（价格）
Price=datasheet.col_values(2)
Price_int=[]
Price_max=0
Price_min=100000000
for i in range(1,len(Price)):
    k=float(Price[i])
    Price_int.append(k)
    if k<Price_min:
        Price_min=k
    if k>Price_max:
        Price_max=k
for i in range(0,len(Price_int)):#归一化处理
    Price_int[i]=2*(Price_int[i]-Price_min)/(Price_max-Price_min)-1
Price_tmp=torch.tensor(Price_int)#将列表转化为张量
price = torch.unsqueeze(Price_tmp,dim=1)#张量扩充维度，适应网络结构

x, price =(Variable(x),Variable(price))#将张量转化为可以反向传播的变量

#构建神经网络
class Net(nn.Module):#创建基类
    def __init__(self):
        super(self).__init__()
        pass
    def forward(self):
        pass
class Net(nn.Module):#定义网络结构
    def __init__(self,n_input,n_hidden,n_output):#构造函数参数为输入维度，隐藏层维度，输出维度
        super(Net,self).__init__()
        self.hidden1 = nn.Linear(n_input,n_hidden)
        self.hidden2 = nn.Linear(n_hidden,n_hidden)
        self.hidden3 = nn.Linear(n_hidden,n_hidden)#三个隐藏层
        self.predict = nn.Linear(n_hidden,n_output)#一个全连接层
    def forward(self,input):#定义网络结构
        out = self.hidden1(input)#第一个隐藏层
        out = F.relu(out)#使用relu作为非线性激活函数
        out = self.hidden2(out)#第二个隐藏层
        out = F.relu(out)#使用relu作为非线性激活函数
        out = self.hidden3(out)#第三个隐藏层
        out = F.relu(out)#使用relu作为非线性激活函数
        out =self.predict(out)#全连接层
        return out

net = Net(34,100,1)#创建网络

optimizer = torch.optim.SGD(net.parameters(),lr = 0.05)#构建优化器（本代码使用梯度下降法），lr为学习率
loss_func = torch.nn.MSELoss()#定义损失函数loss

#学习过程
for t in range(50000):#学习次数为50000次
    prediction = net(x)#进行正向运算
    loss = loss_func(prediction,price)#计算损失函数loss

    optimizer.zero_grad()#重新设置梯度
    loss.backward()#反向传播
    optimizer.step()#更新网络中的参数

    if t%100==0:#随时输出学习进度
        print('t=%s,loss=%s' % (t,loss.data))

#关于调参
#本模型共有两个超参，分别是学习次数和学习率
#第一次尝试学习次数5000，学习率0.1，拟合效果不佳，损失函数未收敛
#将学习次数改为10000，损失函数仍未收敛
#将学习次数改为50000，损失函数收敛，但是在次数超过30000后loss函数下降不明显，学习次数超过40000后loss函数上下波动，基本稳定在0.0006左右，但是出现过拟合现象
#将学习率改为0.01后，损失函数最终稳定在0.001左右，拟合效果不佳
#将学习率改为0.05，损失函数下降为0.07，且过拟合现象减弱
#
# 最终选择学习次数50000，学习率0.05

torch.save(net, 'net.pkl')#保存训练模型

In [ ]:
#本代码为价格预测函数部分
#
#首先调用继承已经训练好的神经网络，在进行数据处理，最后将处理好的输出输入到网络中，获得预测价格
#
#在处理数据的过程中，首先运用学习时获得的极差进行归一化处理，再将归一化处理过后的数据输入到列表中，
#最后将列表转化为张量，再调整矩阵形状适应网络结构
#
#输出预测值的时候先从张量中提取数据，再进行反归一化处理，最终输出预测的价格
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable

#继承上一段代码中构造的神经网络结构
class Net(nn.Module):
    def __init__(self):
        super(self).__init__()
        pass
    def forward(self):
        pass
class Net(nn.Module):
    def __init__(self,n_input,n_hidden,n_output):
        super(Net,self).__init__()
        self.hidden1 = nn.Linear(n_input,n_hidden)
        self.hidden2 = nn.Linear(n_hidden,n_hidden)
        self.hidden3 = nn.Linear(n_hidden,n_hidden)
        self.predict = nn.Linear(n_hidden,n_output)
    def forward(self,input):
        out = self.hidden1(input)
        out = F.relu(out)
        out = self.hidden2(out)
        out = F.relu(out)
        out = self.hidden3(out)
        out = F.relu(out)
        out =self.predict(out)
        return out

#调用网络
netnew=Net(34,100,1)#构建新的网络
netnew = torch.load('net.pkl')#调用训练好的模型

#参数输入（思路相同的仅在第一个参数中作注释）
datas=[]#输入数据储存在名为datas的列表中

k=input("请输入Id")#id被判断为无关变量，舍弃

k=input("请输入Model")#汽车型号被判断为无关变量，舍弃

k=input("请输入Age_08_04")#车龄输入
k=float(k)#pytorch的默认数据类型为浮点
k=(k-1.0)/80.0*2-1#归一化处理（数据来自上一段代码）
if k<-1:#防止出现超过学习数据极值的数据出现
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Mfg_Month")
k=float(k)
k=(k-1.0)/11.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Mfg_Year")
k=float(k)
k=(k-1998.0)/6.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入KM")
k=float(k)
k=(k-1.0)/242999.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Fuel_Type")#输入型号时学习数据中仅有三种类型，观察得Petrol和Diesel为主要，故分别赋值0.5和-0.5，CDG型和其他学习数据中未出现的类型赋值为0
if k=='Diesel':
    datas.append(0.5)
elif k=='Petrol':
    datas.append(-0.5)
else:
    datas.append(0)

k=input("请输入HP")
k=float(k)
k=(k-69.0)/123.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Met_Color")
k=float(k)
k=(k-69.0)/123.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Color")#颜色众多且难以定量为具体数学数据（采用RPG等定量方式过于小题大做），舍弃

k=input("请输入Automatic")
k=float(k)
k=(k-0.0)/1.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入CC")
k=float(k)
k=(k-1300.0)/700.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Doors")
k=float(k)
k=(k-2.0)/3.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Cylinders")#观察得所有数据该项均为4（气缸数几乎为固定值），舍弃

k=input("请输入Gears")
k=float(k)
k=(k-3.0)/3.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Quarterly_Tax")
k=float(k)
k=(k-19.0)/264.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Weight")
k=float(k)
k=(k-1000.0)/615.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Mfg_Guarantee")
k=float(k)
k=(k-0.0)/1.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入BOVAG_Guarantee")
k=float(k)
k=(k-0.0)/1.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入Guarantee_Period")
k=float(k)
k=(k-3.0)/33.0*2-1
if k<-1:
    k=-1
elif k>1:
    k=1
datas.append(k)

k=input("请输入ABS")#此后的数据值仅为0或1，省略归一化处理
datas.append(float(k))

k=input("请输入Airbag_1")
datas.append(float(k))

k=input("请输入Airbag_2")
datas.append(float(k))

k=input("请输入Airco")
datas.append(float(k))

k=input("请输入Automatic_airco")
datas.append(float(k))

k=input("请输入Boardcomputer")
datas.append(float(k))

k=input("请输入CD_Player")
datas.append(float(k))

k=input("请输入Central_Lock")
datas.append(float(k))

k=input("请输入Powered_Windows")
datas.append(float(k))

k=input("请输入Power_Steering")
datas.append(float(k))

k=input("请输入Radio")
datas.append(float(k))

k=input("请输入Mistlamps")
datas.append(float(k))

k=input("请输入Sport_Model")
datas.append(float(k))

k=input("请输入Backseat_Divider")
datas.append(float(k))

k=input("请输入Metallic_Rim")
datas.append(float(k))

k=input("请输入Radio_cassette")
datas.append(float(k))

k=input("请输入Tow_Bar")
datas.append(float(k))

k=input("请输入Validation")
datas.append(float(k))

Datas_tmp=torch.tensor(datas)#将list数据转化为张量
Datas_tm=torch.unsqueeze(Datas_tmp,dim=1)#数据扩充维数，适应网络结构（按照列扩充）
Datas=Datas_tm.permute(1,0)#矩阵转置

prediction=netnew(Datas)#运用训练好的模型获得预测数据

price=prediction.item()#将数据从张量格式中提取出来
prices=(price+1)/2*28150.0+4350.0#反归一化处理（数据来自上一段代码的极差）

print("预测的价格为%s元" % int(prices))#改格式
